In [ ]:
import silence_tensorflow.auto
import tensorflow as tf
gpuDevices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpuDevices))
tf.config.experimental.set_memory_growth(gpuDevices[0], True)
tf.config.optimizer.set_experimental_options({'layout_optimizer': False})
import keras
import os, sys, random, glob, time, logging
from PIL import Image

import transformers
from IPython import display
import datasets
datasets.disable_caching()
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
ts = lambda tensor : keras.preprocessing.image.array_to_img(tensor)

def plot_n_samples(dataset, n ):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        assert tf.is_tensor(dataset[i]), "This ain't a tensor"
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(ts(dataset[i].numpy()))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
# def to_gray(dp):
#     return {'image': 
#         # tf.expand_dims(
#             # tf.image.rgb_to_grayscale(dp['image'])
#             # , axis=-1) 
#             }
    
to_grayscale_pillow = lambda x : {'image' :x['image'].convert("L")}
# to_grayscale_pillow = lambda x:{'image' : tf.expand_dims(tf.image.rgb_to_grayscale(tf.cast(x['image'], dtype=tf.uint8)), axis=2)}

In [ ]:
train_ds = datasets.load_dataset("project-sloth/captcha-images", split='train')
# test_ds = datasets.load_dataset("project-sloth/captcha-images", split='test')
# validation_ds=datasets.load_dataset("project-sloth/captcha-images", split='validation')

In [ ]:
train_dsp = train_ds.map(to_grayscale_pillow).with_format("tf")

In [ ]:
plot_n_samples(train_dsp['image'], 4)
# print(train_dsp[0]['image'].shape)

In [ ]:
# HYPs


batch_size = 32;
lr = 0.0001;
epochs = 10;

In [ ]:
def make_encoder() :
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(50,200,1)))
    
    model.add(keras.layers.Conv2D(32,(3,3), activation='relu', padding='same'))
    model.add(keras.layers.MaxPool2D((2,2)))

    assert model.output_shape == (None, 25,100,32)

    model.add(keras.layers.Conv2D(64,(3,3), activation='relu', padding='same'))
    model.add(keras.layers.MaxPool2D((2,2)))

    
    model.add(keras.layers.Conv2D(128,(3,3), activation='relu', padding='same'))
    model.add(keras.layers.MaxPool2D((2,2)))
    model.add(keras.layers.Flatten())

    print(model.output_shape)

    model.add(keras.layers.Dense(128,activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64, activation='relu'))

    print(model.output_shape)    

    for i in range(5):
        model.add(keras.layers.Dense(10,activation='softmax', name=f'digit{i+1}'))
    

    return model
    

In [ ]:
model = make_encoder()

In [ ]:
model.summary()

In [ ]:
_ = model(train_ds['image'][0])

In [ ]:
outputs = [model.get_layer(f'digit{i+1}').output for i in range(5)]
model = tf.keras.Model(inputs=model.input, outputs=outputs)